### ECE 899 Independent Study
#### Libo Zhang, Duke University

In [1]:
import os
import sys
import torch
import itertools
import cv2 as cv
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline

from sklearn import metrics
from torchsummary import summary
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

from train import *
from models import *
from utilities import *

In [2]:
X_train_list, X_valid_list = [], []
torch.set_printoptions(profile = "full", precision = 3)
train_list = build_all_training_file_list(N_normal = 80,   N_tumor = 55)
temp_list  = build_all_training_file_list(N_normal = 160,  N_tumor = 111)
valid_list = [x for x in temp_list if x not in train_list]
device     = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(torch.cuda.current_device()))

NVIDIA GeForce RTX 2070 with Max-Q Design


In [ ]:
epochs = 100
cut_epochs = 50
MA = Attention_Alpha().to(device)
MB = Attention_Beta().to(device)
ME = ResNet_Encoder().to(device)
MT = Basic_Transformer(out_chs = 32).to(device)
MC = Classifier(in_channels = 128, cut_epochs = cut_epochs).to(device)
criterion  = nn.BCELoss()
optimizer1 = optim.Adam(list(MA.parameters()) + \
                        list(MB.parameters()) + \
                        list(ME.parameters()) + \
                        list(MC.parameters()), lr = 1e-4)
optimizer2 =      optim.Adam(MT.parameters(),  lr = 1e-4)
scheduler  =      optim.lr_scheduler.StepLR(optimizer1, 
                                            step_size = 10, gamma = 0.5)

A, B, C, D, E, F, G, H = Train_Models(MA, MB, ME, MT, MC, 
                    train_list, valid_list, X_train_list, X_valid_list, 
                    criterion, optimizer1, optimizer2, scheduler, 
                    epochs = epochs, cut_epochs = cut_epochs, 
                    h1 = 3200, w1 = 3200, h2 = 80, w2 = 80, 
                    s1 = 0.03125, s2 = 0.125, N = 50)
np.save("train_loss", A)
np.save("train_auc",  B)
np.save("train_acc",  C)
np.save("train_dss",  D)
np.save("valid_loss", E)
np.save("valid_auc",  F)
np.save("valid_acc",  G)
np.save("valid_dss",  H)